<a href="https://colab.research.google.com/github/NehaSontakk/BATH-Prokka-Comparison/blob/main/Update_1_of_BATH_rerun_E_value_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install biopython

!sudo apt-get update
!sudo apt-get install hmmer

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
hmmer is already the newest version (3.3.2+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [10]:
from Bio import SeqIO
import pandas as pd
import re
import os
import subprocess
import pandas as pd

In [11]:
df = pd.read_excel("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Organized/Alignment_Overlaps_Unique_Bedmap_Output.xlsx")
dna_fasta_file = "bin_82_prokka_annotation.fna"

In [12]:
#Extract the information for each row of information
def extract_info(row):
    if pd.notnull(row['BATH']):
        bath_data = re.split(r'\t|\s+', str(row['BATH']))
        #print(bath_data)
        row['BATH_contig'] = bath_data[0]
        if len(bath_data) >= 3:
            row['BATH_annot'] = bath_data[-3]
            if int(bath_data[1]) < int(bath_data[2]):
              row['BATH_start'] = abs(int(bath_data[1]))
              row['BATH_end'] = abs(int(bath_data[2]))
            else:
              row['BATH_end'] = abs(int(bath_data[1]))
              row['BATH_start'] = abs(int(bath_data[2]))
        else:
            row['BATH_annot'] = None

    if pd.notnull(row['Prokka']):
        prokka_data = re.split(r'\t|\s+', str(row['Prokka']))
        prokka_annotr = re.split(r':', str(row['Prokka']))[-1]
        row['Prokka_contig'] = prokka_data[0]
        if len(prokka_data) >= 3:
            #print("Prokka starts:",int(prokka_data[1]))
            row['Prokka_annot'] = prokka_annotr
            if int(prokka_data[1]) < int(prokka_data[2]):
              row['Prokka_start'] = abs(int(prokka_data[1]))
              row['Prokka_end'] = abs(int(prokka_data[2]))
            else:
              row['Prokka_end'] = abs(int(prokka_data[1]) )
              row['Prokka_start'] = abs(int(prokka_data[2]))
        else:
            row['Prokka_annot'] = None
    return row

In [13]:
df2 = df.apply(extract_info, axis=1)

In [14]:
#prokka_only = df[df['label']=='Prokka Annotated']

## Get BATH E-values

In [15]:
bath_protein = pd.read_excel("/content/bin82_BATH_deduplicated-2.xlsx")
unnamed_cols = [col for col in bath_protein.columns if col.startswith('Unnamed')]
bath_protein.drop(columns=unnamed_cols, inplace=True)

In [16]:
df3 = pd.merge(df2,bath_protein[['target name','query name','E-value','shifts']],how="left",left_on=['BATH_contig','BATH_annot'],right_on=['target name','query name'])

## Get Prokka E-values

1.   Get Prokka protein sequence
  1. HAMAP sequences need to be obtained seperately. Through the .hmm file
  2. Uniprot and other sequences need to be obtained through the db files
2.   Get Prokka DNA sequence
3.   Spot check
4.   Convert to fasta/ fa files required for BATH
5.   Run BATH on the files


In [17]:
#Adding Prokka DNA sequence
def prokka_genome_data(row):
  if pd.isna(row['Prokka_annot'] or row['Prokka_start']) or pd.isna(row['Prokka_end']):
        return None
  print(row['Prokka_contig'])
  print(row['Prokka_annot'])
  start = int(row['Prokka_start'])
  print("Prokka starts:",start)
  end = int(row['Prokka_end'])
  print("Prokka end:",end)
  dna_fasta_file = "/content/bin_82_prokka_annotation.fna"
  for record in SeqIO.parse(dna_fasta_file, "fasta"):
      if record.id == row['Prokka_contig']:
        print(len(record.seq))
        sub_sequence = record.seq[start-1:end]
        #print(start-end)
        #print(len(str(sub_sequence)))
        return str(sub_sequence)

In [18]:
df3['Prokka DNA Sequence'] = df3.apply(prokka_genome_data,axis=1)

k127_486726
002006
Prokka starts: 2241
Prokka end: 4844
181032
k127_486726
002006
Prokka starts: 26042
Prokka end: 26653
181032
k127_486726
002006
Prokka starts: 51898
Prokka end: 52266
181032
k127_486726
002006
Prokka starts: 56798
Prokka end: 58150
181032
k127_486726
002006
Prokka starts: 63786
Prokka end: 64232
181032
k127_486726
002006
Prokka starts: 82838
Prokka end: 84010
181032
k127_486726
002006
Prokka starts: 90960
Prokka end: 91436
181032
k127_486726
002006
Prokka starts: 98312
Prokka end: 99532
181032
k127_486726
002006
Prokka starts: 103857
Prokka end: 104165
181032
k127_486726
002006
Prokka starts: 113844
Prokka end: 115412
181032
k127_486726
002006
Prokka starts: 118184
Prokka end: 118432
181032
k127_486726
002006
Prokka starts: 122980
Prokka end: 123543
181032
k127_486726
002006
Prokka starts: 123856
Prokka end: 124935
181032
k127_486726
002006
Prokka starts: 124939
Prokka end: 125523
181032
k127_486726
002006
Prokka starts: 131696
Prokka end: 132352
181032
k127_486726
0

In [19]:
#Finding the Prokka protein
import os

file_paths = [
    "/content/drive/MyDrive/Lab Work/Parkinsons_Data/Prokka db/kingdom/Bacteria/sprot",
    "/content/drive/MyDrive/Lab Work/Parkinsons_Data/Prokka db/kingdom/Bacteria/IS",
    "/content/drive/MyDrive/Lab Work/Parkinsons_Data/Prokka db/kingdom/Bacteria/AMR",
    "/content/drive/MyDrive/Lab Work/Parkinsons_Data/Prokka db/kingdom/Archaea/sprot",
    "/content/drive/MyDrive/Lab Work/Parkinsons_Data/Prokka db/kingdom/Mitochondria/sprot",
    "/content/drive/MyDrive/Lab Work/Parkinsons_Data/Prokka db/kingdom/Viruses/sprot"
]
combined_file = "/content/drive/MyDrive/Lab Work/Parkinsons_Data/combined_sequences.fasta"

with open(combined_file, 'w') as combined_fasta:
    for file_path in file_paths:
        if os.path.exists(file_path):
            for record in SeqIO.parse(file_path, "fasta"):
                SeqIO.write(record, combined_fasta, "fasta")
        else:
            print(f"File not found: {file_path}")

def get_fasta_entries(fasta_path, protein_name):
    #print(protein_name)
    if pd.isna(protein_name):
          return None
    for record in SeqIO.parse(fasta_path, "fasta"):
        if protein_name == record.name:
          #print(record.seq)
          return str(record.seq)

def get_prokka_fasta_sequence(row):
    return get_fasta_entries(combined_file, row['Prokka_annot'])



In [20]:
import numpy as np
df3['Prokka_annot'] = df3['Prokka_annot'].str.replace("002006", "", regex=False)
df3['Prokka_annot'].replace("", np.nan, inplace=True)

In [21]:
df3['Prokka Uniprot Protein Sequence'] = df3.apply(get_prokka_fasta_sequence,axis=1)

In [22]:
df3[~df3['Prokka_annot'].isna()]['Prokka Uniprot Protein Sequence']

42                                                  None
54                                                  None
99                                                  None
114                                                 None
155                                                 None
                             ...                        
991    MIEFVYPHTQLVAGVDEVGRGPLVGAVVTAAVILDPARPIAGLNDS...
992    MKQDSRFPNLFILDHPLIQHKLTHMRDKDTSTRTFRELLREITLLM...
994    MNETPKPNSFRSGPDEDGRFGIYGGRFVAETLMPLILDLQDEWNKA...
995    MTDVDRVEPETPPEREETDAELTARFERDAIPLLDQLYGGALRMTR...
996    MSGIDAKKIRTRHFREAKVNGQKVSVLTSYDALSARIFDEAGVDML...
Name: Prokka Uniprot Protein Sequence, Length: 471, dtype: object

## Get BATH fasta sequence

In [23]:
def get_bath_fasta_sequence(row):
    return get_fasta_entries(combined_file, row['BATH_annot'])

In [24]:
df3['BATH Uniprot Protein Sequence'] = df3.apply(get_bath_fasta_sequence,axis=1)

## Running BATH with HAMAP hmm data

In [25]:
!git clone https://github.com/TravisWheelerLab/BATH
%cd BATH
!git clone https://github.com/TravisWheelerLab/easel
%cd easel
!git checkout BATH
%cd ..
!autoconf
!./configure
!make


Cloning into 'BATH'...
remote: Enumerating objects: 17550, done.
remote: Counting objects: 100% (1851/1851), done.
remote: Compressing objects: 100% (688/688), done.
remote: Total 17550 (delta 1291), reused 1693 (delta 1160), pack-reused 15699
Receiving objects: 100% (17550/17550), 65.61 MiB | 19.28 MiB/s, done.
Resolving deltas: 100% (13362/13362), done.
/content/BATH
Cloning into 'easel'...
remote: Enumerating objects: 9372, done.
remote: Counting objects: 100% (713/713), done.
remote: Compressing objects: 100% (343/343), done.
remote: Total 9372 (delta 383), reused 651 (delta 370), pack-reused 8659
Receiving objects: 100% (9372/9372), 18.45 MiB | 13.69 MiB/s, done.
Resolving deltas: 100% (7005/7005), done.
/content/BATH/easel
Branch 'BATH' set up to track remote branch 'BATH' from 'origin'.
Switched to a new branch 'BATH'
/content/BATH
configure: Configuring BATH for your system.
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
check

In [29]:
#RUN for single seq list

import os
import subprocess
from Bio import SeqIO

# Function to fetch HMMs and convert to BATH's format
def fetch_hmm_path(protein_name, hmm_db_path):
    hmm_output_path = f"/content/output_{protein_name}.hmm"
    bhmm_output_path = f"/content/{protein_name}.bhmm"
    fetch_command = f"hmmfetch -o '{hmm_output_path}' '{hmm_db_path}' '{protein_name}'"
    os.system(fetch_command)
    bath_convert_path = "/content/BATH/src/bathconvert"
    convert_command = f"{bath_convert_path} {bhmm_output_path} {hmm_output_path}"
    os.system(convert_command)
    if os.path.exists(hmm_output_path):
        os.remove(hmm_output_path)
    return bhmm_output_path

# Function to run bathsearch on a whole DNA sequence file for a list of HMMs
def run_bathsearch_on_dna(dna_sequence_path, protein_list, hmm_db_path):
    # Read the whole DNA sequence
    with open(dna_sequence_path, 'r') as file:
        dna_sequence = file.read()

    results = {}
    for protein_name in protein_list:
        if protein_name.startswith('MF_'):
            bhmm_output_path = fetch_hmm_path(protein_name, hmm_db_path)
            out_path = f"/content/{protein_name}_results.out"
            tbl_out_path = f"/content/{protein_name}_results.tbl"

            command = f"/content/BATH/src/bathsearch --tblout '{tbl_out_path}' -o '{out_path}' '{bhmm_output_path}' '{dna_sequence_path}'"
            print("Running command:", command)
            result = subprocess.run(command, shell=True, capture_output=True, text=True)

            if result.returncode != 0:
                print(f"Error running bathsearch for {protein_name}: {result.stderr}")
            else:
                print(f"Results saved to {out_path} and {tbl_out_path}")

            try:
                with open(out_path, 'r') as file:
                    output_results = file.read()
                    results[protein_name] = output_results
                    #os.remove(out_path)
            except FileNotFoundError:
                results[protein_name] = "Output file not found."
            os.remove(bhmm_output_path)

    return results

# Path to the whole DNA sequence
dna_sequence_path = "/content/bin_82_prokka_annotation.fna"
protein_list = ['MF_00671', 'MF_01498','MF_00948','MF_00911','MF_01531','MF_00227','MF_02048','MF_00463','MF_01916','MF_01259']
hmm_db_path = "/content/drive/MyDrive/Lab Work/Parkinsons_Data/Prokka db/hmm/HAMAP.hmm"

# Run the bathsearch for each protein on the whole DNA sequence
results = run_bathsearch_on_dna(dna_sequence_path, protein_list, hmm_db_path)
print(results)


Running command: /content/BATH/src/bathsearch --tblout '/content/MF_00671_results.tbl' -o '/content/MF_00671_results.out' '/content/MF_00671.bhmm' '/content/bin_82_prokka_annotation.fna'
Results saved to /content/MF_00671_results.out and /content/MF_00671_results.tbl
Running command: /content/BATH/src/bathsearch --tblout '/content/MF_01498_results.tbl' -o '/content/MF_01498_results.out' '/content/MF_01498.bhmm' '/content/bin_82_prokka_annotation.fna'
Results saved to /content/MF_01498_results.out and /content/MF_01498_results.tbl
Running command: /content/BATH/src/bathsearch --tblout '/content/MF_00948_results.tbl' -o '/content/MF_00948_results.out' '/content/MF_00948.bhmm' '/content/bin_82_prokka_annotation.fna'
Results saved to /content/MF_00948_results.out and /content/MF_00948_results.tbl
Running command: /content/BATH/src/bathsearch --tblout '/content/MF_00911_results.tbl' -o '/content/MF_00911_results.out' '/content/MF_00911.bhmm' '/content/bin_82_prokka_annotation.fna'
Results s

In [31]:
import pandas as pd
import glob

# Path where the .tbl files are stored
tbl_files_path = "/content/*.tbl"

# List all .tbl files
tbl_files = glob.glob(tbl_files_path)

# Define column names for the DataFrame
col_names = ['target name', 'accession', 'query name', 'accession1', 'hmm len', 'hmm from', 'hmm to', 'seq len', 'ali from', 'ali to', 'env from', 'env to', 'E-value', 'score', 'bias', 'shifts', 'stops', 'pipe', 'description of target', 'extra']

# Initialize an empty list to store DataFrames
dfs = []

# Loop over each file and process it
for file_path in tbl_files:
    bathout = pd.read_table(file_path, sep="\s+", header=None, skiprows=2, skipfooter=8, engine='python')
    bathout.rename(columns=dict(zip(bathout.columns, col_names)), inplace=True)
    dfs.append(bathout)

# Concatenate all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to an Excel file
excel_output_path = "/content/HAMAP_run2_bath_output.xlsx"
combined_df.to_excel(excel_output_path, index=False)

print(f"Combined file saved as {excel_output_path}")


Combined file saved as /content/HAMAP_run2_bath_output.xlsx


In [18]:
#HAMAP
#TO DO
#get the hmm files
#get the indices for rows that start with MF
#get MF names
#for each MF name we gather the real hmm from the .hmm file

def fetch_hmm_path(protein_name, hmm_db_path):
    hmm_output_path = f"/content/output_{protein_name}.hmm"
    bhmm_output_path = f"/content/{protein_name}.bhmm"
    fetch_command = f"hmmfetch -o '{hmm_output_path}' '{hmm_db_path}' '{protein_name}'"
    os.system(fetch_command)
    bath_convert_path = "/content/BATH/src/bathconvert"
    convert_command = f"{bath_convert_path} {bhmm_output_path} {hmm_output_path}"
    os.system(convert_command)
    if os.path.exists(hmm_output_path):
        os.remove(hmm_output_path)

    return bhmm_output_path

def run_bathsearch(row, bhmm_db_path):
    protein_name = row['Prokka_annot']
    if pd.isna(protein_name) :
          return None

    if not row['Prokka_annot'].startswith('MF_'):
      return None

    dna_sequence_path = f"/content/{protein_name}_dna_sequence.fasta"

    # Write the DNA sequence to a file
    with open(dna_sequence_path, 'w') as f:
        f.write(f">{protein_name}\n{row['Prokka DNA Sequence']}\n")

    bhmm_output_path = fetch_hmm_path(protein_name, hmm_db_path)
    out_path = f"/content/{protein_name}_results.out"
    tbl_out_path = f"/content/{protein_name}_results.tbl"

    command = f"/content/BATH/src/bathsearch -o '{out_path}' '{bhmm_output_path}' '{dna_sequence_path}'"

    print("Running command:", command)  # Debugging line to see the command
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"Error running bathsearch for {protein_name}: {result.stderr}")
    else:
        print(f"Results saved to {out_path} and {tbl_out_path}")

    # Clean up DNA sequence file
    os.remove(dna_sequence_path)
    os.remove(bhmm_output_path)

    # Reading the output from the generated output file
    try:
        with open(out_path, 'r') as file:
            output_results = file.read()
            os.remove(out_path)
            return output_results
    except FileNotFoundError:
        return "Output file not found."

# Assuming bhmm_db_path is set to the directory path, not the HMM file itself
bhmm_db_path = "/content/"
hmm_db_path = "/content/drive/MyDrive/Lab Work/Parkinsons_Data/Prokka db/hmm/HAMAP.hmm"
#filtered_df['prokka_hamap_protein'] = filtered_df['Prokka_annot'].apply(lambda x: fetch_hmm_path(x, hmm_db_path))
df3['Prokka_wbathsearch_HAMAP'] = df3.apply(run_bathsearch, axis=1, args=(bhmm_db_path,))


def extract_e_value_from_output(bath_output):
    """Extract the E-value from the bathsearch output text, specifically the line after '---'."""
    if pd.isna(bath_output) :
       return None


    lines = bath_output.split('\n')
    capture_next = False  # Flag to indicate capturing the next line's E-value

    for line in lines:
        if '---' in line:  # Detect the separator line
            capture_next = True
        elif capture_next and line.strip() and not line.startswith('#'):
            # Extract from the next non-empty, non-comment line after the '---'
            parts = line.split()
            if len(parts) > 0:
                return parts[0]  # Return the E-value, which is expected to be the first element
            capture_next = False  # Reset the flag if no valid data is found immediately after the '---'

    return "E-value not found"

df3['Prokka_wbathsearch_HAMAP_E-value'] = df3['Prokka_wbathsearch_HAMAP'].apply(extract_e_value_from_output)

Running command: /content/BATH/src/bathsearch -o '/content/MF_00671_results.out' '/content/MF_00671.bhmm' '/content/MF_00671_dna_sequence.fasta'
Results saved to /content/MF_00671_results.out and /content/MF_00671_results.tbl
Running command: /content/BATH/src/bathsearch -o '/content/MF_00073_results.out' '/content/MF_00073.bhmm' '/content/MF_00073_dna_sequence.fasta'
Results saved to /content/MF_00073_results.out and /content/MF_00073_results.tbl
Running command: /content/BATH/src/bathsearch -o '/content/MF_00530_results.out' '/content/MF_00530.bhmm' '/content/MF_00530_dna_sequence.fasta'
Results saved to /content/MF_00530_results.out and /content/MF_00530_results.tbl
Running command: /content/BATH/src/bathsearch -o '/content/MF_01351_results.out' '/content/MF_01351.bhmm' '/content/MF_01351_dna_sequence.fasta'
Results saved to /content/MF_01351_results.out and /content/MF_01351_results.tbl
Running command: /content/BATH/src/bathsearch -o '/content/MF_01430_results.out' '/content/MF_01

## Running BATH with Prokka Uniprot Sequences

In [27]:
def run_bathsearch(row, output_dir):
    protein_name = row.get('Prokka_annot', None)

    # Debugging: Print the protein name to check it's being correctly read
    print(f"Processing protein: {protein_name}")

    if pd.isna(protein_name):
        print("Invalid or missing protein name.")
        return None

    if row['Prokka_annot'].startswith('MF_'):
      return None

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Define file paths
    dna_sequence_path = os.path.join(output_dir, f"{protein_name}_dna_sequence.fasta")
    protein_sequence_path = os.path.join(output_dir, f"{protein_name}.fa")
    out_path = os.path.join(output_dir, f"{protein_name}_results.out")

    # Write the sequences to files
    with open(dna_sequence_path, 'w') as dna_file, open(protein_sequence_path, 'w') as protein_file:
        dna_file.write(f">{protein_name}\n{row.get('Prokka DNA Sequence', '')}\n")
        protein_file.write(f">{protein_name}\n{row.get('Prokka Uniprot Protein Sequence', '')}\n")

    # Construct and print command
    command = f"/content/BATH/src/bathsearch -o '{out_path}' '{protein_sequence_path}' '{dna_sequence_path}'"
    print("Running command:", command)
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    # Check results
    if result.returncode != 0:
        print(f"Error running bathsearch: {result.stderr}")
        return None

    # Attempt to read output file
    try:
        with open(out_path, 'r') as file:
            output_results = file.read()
            print(output_results)
            os.remove(out_path)
            os.remove(dna_sequence_path)
            #os.remove(protein_sequence_path)
            return output_results
    except FileNotFoundError:
        print("Output file not found.")
        return "Output file not found."


output_dir = "/content"
df_name = 'df3'
if df_name in globals():
    globals()[df_name]['Prokka_wbathsearch_UniProt'] = globals()[df_name].apply(run_bathsearch, axis=1, args=(output_dir,))
    print(globals()[df_name])
else:
    print(f"DataFrame {df_name} does not exist.")


Streaming output truncated to the last 5000 lines.
               l    p    +    i    y              g    +    +         +    e    r    +    e    r              +         l    e    +    +       
               L    P    L    I    Y    r    K    G    V    S    A    R    E    R    K    E    R    V    A    Q    V    L    E    R    L    E      
  O31711 322  CTG  CCG  CTG  ATT  TAC  CGC  AAG  GGC  GTC  AGC  GCC  CGC  GAG  CGC  AAG  GAG  CGC  GTG  GCG  CAG  GTA  CTT  GAA  CGC  CTC  GAG  399
               *    *    *    *    *    5    6    *    *    *    *    *    *    *    *    *    *    *    *    *    *    *    *    *    *    *   PP

  O31711 131   l    a    d    r    m    l    h    m    p    n    e    l    s    g    g    q    k    q    r    v    a    i    a    r    a    i   156
               +              r    +         h    +    p    +    +    l    s    g    g    q         q    r    v    a    i    a    r    a    i  
               M    T    H    R    I    H    H    L    P    S    Q   

In [20]:
df3['Prokka_wbathsearch_UniProt_Evalue'] = df3['Prokka_wbathsearch_UniProt'].apply(extract_e_value_from_output)

In [21]:
df3.to_excel("/content/Test_set5.xlsx")

In [22]:
filtered_df = df3[df3['Prokka_annot'].str.startswith('MF_', na=False)]

# Running Phmmer with HAMAP and Uniprot

In [28]:
bathout = pd.read_table("/content/NewBATH_7_output110624.tbl",sep="\s+",header=None,skiprows=2,skipfooter=8)
col_names = ['target name', 'accession', 'query name', 'accession1', 'hmm len', 'hmm from', 'hmm to', 'seq len', 'ali from', 'ali to', 'env from', 'env to', 'E-value', 'score', 'bias', 'shifts', 'stops', 'pipe', 'description of target', 'extra']
bathout.rename(columns=dict(zip(bathout.columns, col_names)), inplace=True)

<ipython-input-28-caaee3755430>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  bathout = pd.read_table("/content/NewBATH_7_output110624.tbl",sep="\s+",header=None,skiprows=2,skipfooter=8)


In [29]:
bathout

,target name,accession,query name,accession1,hmm len,hmm from,hmm to,seq len,ali from,ali to,env from,env to,E-value,score,bias,shifts,stops,pipe,description of target
0,k127_1013475,-,P11551,-,438,17,175,183529,96684,97175,96672,97196,2.500000e-05,22.3,8.6,0,0,std,-
1,k127_1013475,-,P11551,-,438,351,422,183529,97698,97919,97647,97964,2.300000e-01,9.3,0.6,0,0,std,-
2,k127_486726,-,P12758,-,253,7,135,181032,117238,117660,117226,117678,2.000000e-06,28.3,0.0,0,0,std,-
3,k127_486726,-,P12758,-,253,154,224,181032,117769,117984,117700,118005,3.000000e+00,8.1,0.0,0,0,std,-
4,k127_533492,-,P42979,-,111,2,103,131788,100642,100944,100639,100962,4.200000e-09,39.1,0.0,0,0,std,-
5,k127_748425,-,P55192,-,250,73,202,125518,122873,123256,122807,123283,1.300000e-06,29.7,0.0,0,0,std,-
6,k127_39946,-,P55192,-,250,99,154,74930,62594,62764,62573,62782,3.800000e-01,11.7,0.0,0,0,std,-
7,k127_922904,-,P76352,-,547,96,531,133853,108091,109428,108073,109461,1.200000e-10,41.8,39.0,0,0,std,-
8,k127_533492,-,P76352,-,547,95,509,131788,77263,76031,77287,75968,3.600000e-09,37.0,27.9,0,0,std,-
9,k127_1013475,-,P76352,-,547,97,484,183529,112453,113583,112429,113694,3.800000e-08,33.6,17.9,0,0,std,-


In [33]:
bathout.to_excel("/content/zddhd.xlsx")

In [ ]:
'''
import os
import subprocess
import pandas as pd

def fetch_onlyhmm_path(protein_name, hmm_db_path):
    hmm_output_path = f"/content/output_{protein_name}.hmm"
    fetch_command = f"hmmfetch -o '{hmm_output_path}' '{hmm_db_path}' '{protein_name}'"
    result = os.system(fetch_command)
    if result != 0:
        print(f"Failed to fetch HMM for {protein_name} from {hmm_db_path}")
        return None
    if os.path.getsize(hmm_output_path) == 0:  # Check if the file is empty
        print(f"Warning: Generated HMM file {hmm_output_path} is empty.")
        return None
    return hmm_output_path

def run_phmmer_HAMAP_Uniprot(row, output_dir, hmm_db_path):
    if pd.isna(row['Prokka_annot']) or pd.isna(row['BATH_annot']):
        return None
    print(f"Processing {row['Prokka_annot']} and {row['BATH_annot']}")

    if row['Prokka_annot'].startswith('MF_') or row['BATH_annot'].startswith('MF_'):
        prokka_hmm_output_path = fetch_onlyhmm_path(row['Prokka_annot'], hmm_db_path) if row['Prokka_annot'].startswith('MF_') else None
        bath_hmm_output_path = fetch_onlyhmm_path(row['BATH_annot'], hmm_db_path) if row['BATH_annot'].startswith('MF_') else None

        if not prokka_hmm_output_path or not bath_hmm_output_path:
            print(f"Skipping due to missing HMM file for {'Prokka' if not prokka_hmm_output_path else 'Bath'}")
            return None

        out_path = os.path.join(output_dir, f"{row['Prokka_annot']}_{row['BATH_annot']}_results.out")
        cmd = ['phmmer', '--tblout', out_path, prokka_hmm_output_path, bath_hmm_output_path]
        result = subprocess.run(cmd, capture_output=True, text=True)

        if result.returncode != 0:
            print(f"Error running phmmer: {result.stderr}")
            return None

        return result.stdout

    return "No valid annotations to process"

# Example usage
output_dir = "/content/"
df3['phmmeroutput'] = df3.apply(lambda row: run_phmmer_HAMAP_Uniprot(row, output_dir, hmm_db_path), axis=1)
'''


In [ ]:
'''
def fetch_onlyhmm_path(protein_name, hmm_db_path):
    hmm_output_path = f"/content/output_{protein_name}.hmm"
    fetch_command = f"hmmfetch -o '{hmm_output_path}' '{hmm_db_path}' '{protein_name}'"
    result = os.system(fetch_command)
    if result != 0:
        print(f"Failed to fetch HMM for {protein_name} from {hmm_db_path}")
        return None
    if os.path.getsize(hmm_output_path) == 0:  # Check if the file is empty
        print(f"Warning: Generated HMM file {hmm_output_path} is empty.")
        return None
    return hmm_output_path



def run_phmmer_HAMAP_Uniprot(row,output_dir):
  out_path = None
  result = None
  if pd.isna(row['Prokka_annot']) or pd.isna(row['BATH_annot']):
        return None
  print(row['Prokka_annot'],row['BATH_annot'])
  if row['Prokka_annot'].startswith('MF_') or row['BATH_annot'].startswith('MF_'):
      if row['Prokka_annot'].startswith('MF_'):
        print(row['BATH Uniprot Protein Sequence'])
        out_path = os.path.join(output_dir, f"{row['Prokka_annot']}_{row['BATH_annot']}_results.out")
        #Fetch the HAMAP hmm file for this
        prokka_hmm_output_path = fetch_onlyhmm_path(row['Prokka_annot'], hmm_db_path)
        print(prokka_hmm_output_path)
        #Make a second protein file for the bath sequence
        protein_sequence_path = os.path.join(output_dir, f"{row['BATH_annot']}_forphmmer.fa")
        with open(protein_sequence_path, 'w') as protein_file:
            protein_file.write(f">{row['BATH_annot']}\n{row['BATH Uniprot Protein Sequence']}\n")
        cmd = ['phmmer', '--tblout', out_path, protein_sequence_path, prokka_hmm_output_path]
        result = subprocess.run(cmd, capture_output=True, text=True)
        #os.remove(protein_sequence_path)
        #os.remove(prokka_hmm_output_path)
        if prokka_hmm_output_path is None:
          print(f"Skipping phmmer due to missing HMM file for {'Prokka' if prokka_hmm_output_path is None else 'Bath'}")
          return None

        if result.returncode != 0:
          return f"Error running phmmer: {result.stderr}"
        return result.stdout

      elif row['BATH_annot'].startswith('MF_'):
        #Fetch the HAMAP file for BATH
        bath_hmm_output_path = fetch_onlyhmm_path(row['BATH_annot'], hmm_db_path)
        out_path = os.path.join(output_dir, f"{row['Prokka_annot']}_{row['BATH_annot']}_results.out")
        protein_sequence_path = os.path.join(output_dir, f"{row['Prokka_annot']}_forphmmer.fa")
        with open(protein_sequence_path, 'w') as protein_file:
            protein_file.write(f">{row['Prokka_annot']}\n{row.get('Prokka Uniprot Protein Sequence', '')}\n")
        cmd = ['phmmer', '--tblout', out_path, protein_sequence_path, bath_hmm_output_path]
        result = subprocess.run(cmd, capture_output=True, text=True)
        #os.remove(protein_sequence_path)
        #os.remove(prokka_hmm_output_path)

        if result.returncode != 0:
          return f"Error running phmmer: {result.stderr}"
        return result.stdout

        if bath_hmm_output_path is None:
                print(f"Skipping phmmer due to missing HMM file for {'Prokka' if prokka_hmm_output_path is None else 'Bath'}")
                return None

  if row['Prokka_annot'].startswith('MF_') and row['BATH_annot'].startswith('MF_'):
      #print("Two MFs")
      out_path = os.path.join(output_dir, f"{row['Prokka_annot']}_{row['BATH_annot']}_results.out")
      bath_hmm_output_path = fetch_onlyhmm_path(row['BATH_annot'], hmm_db_path)
      prokka_hmm_output_path = fetch_onlyhmm_path(row['Prokka_annot'], hmm_db_path)
      cmd = ['phmmer', '--tblout', out_path, bath_hmm_output_path, prokka_hmm_output_path]
      result = subprocess.run(cmd, capture_output=True, text=True)
      os.remove(protein_sequence_path)

  if not row['Prokka_annot'].startswith('MF_') or not row['BATH_annot'].startswith('MF_'):
      out_path = os.path.join(output_dir, f"{row['Prokka_annot']}_{row['BATH_annot']}_results.out")
      prokka_protein_sequence_path = os.path.join(output_dir, f"{row['Prokka_annot']}_forphmmer.fa")
      with open(prokka_protein_sequence_path, 'w') as protein_file:
          protein_file.write(f">{row['Prokka_annot']}\n{row.get('Prokka Uniprot Protein Sequence', '')}\n")
      bath_protein_sequence_path = os.path.join(output_dir, f"{row['BATH_annot']}_forphmmer.fa")
      with open(bath_protein_sequence_path, 'w') as protein_file:
          protein_file.write(f">{row['BATH_annot']}\n{row.get('BATH Uniprot Protein Sequence', '')}\n")
      cmd = ['phmmer', '--tblout', out_path, bath_protein_sequence_path, prokka_protein_sequence_path]
      result = subprocess.run(cmd, capture_output=True, text=True)
      try:
        # File operations that might fail if paths are incorrect or files do not exist
        if os.path.exists(prokka_protein_sequence_path):
            os.remove(prokka_protein_sequence_path)
        if os.path.exists(bath_protein_sequence_path):
            os.remove(bath_protein_sequence_path)
      except Exception as e:
          print(f"Error removing files: {e}")

  e_value, i_evalue, c_evalue = "No match", "No match", "No match"
  found_match = False

  if result and result.returncode == 0:
      with open(out_path, 'r') as file:
          for line in file:
              if not line.startswith("#"):
                  found_match = True
                  parts = line.strip().split()
                  if len(parts) > 11:  # Verify the column count
                      e_value = parts[4]  # Typical E-value column
                      print(e_value)
                      i_evalue = parts[11]  # Typical i-Evalue column
                      c_evalue = parts[10]  # Typical c-Evalue column
                      os.remove(out_path)
                      break
  elif result:
      print(f"Error running phmmer: {result.stderr}")

  if not found_match:
      os.remove(out_path)
      return "No significant matches found", "No significant matches found", "No significant matches found"

  return e_value, i_evalue, c_evalue



output_dir = "/content/"
#df3.apply(lambda row: run_phmmer_HAMAP_Uniprot(row), axis=1)
df3['phmmeroutput'] = df3.apply(run_phmmer_HAMAP_Uniprot, axis=1, args=(output_dir,))
'''

In [ ]:
'''
import os
import pandas as pd
import subprocess

def fetch_onlyhmm_path(protein_name, hmm_db_path):
    hmm_output_path = f"/content/output_{protein_name}.hmm"
    fetch_command = f"hmmfetch -o '{hmm_output_path}' '{hmm_db_path}' '{protein_name}'"
    result = os.system(fetch_command)
    if result != 0:
        print(f"Failed to fetch HMM for {protein_name} from {hmm_db_path}")
        return None
    if os.path.getsize(hmm_output_path) == 0:  # Check if the file is empty
        print(f"Warning: Generated HMM file {hmm_output_path} is empty.")
        return None
    return hmm_output_path

def run_phmmer_HAMAP_Uniprot(row, output_dir, hmm_db_path):
    out_path = None
    result = None
    if pd.isna(row['Prokka_annot']) or pd.isna(row['BATH_annot']):
        return None
    print(row['Prokka_annot'], row['BATH_annot'])

    try:
        if row['Prokka_annot'].startswith('MF_') or row['BATH_annot'].startswith('MF_'):
            if row['Prokka_annot'].startswith('MF_'):
                prokka_hmm_output_path = fetch_onlyhmm_path(row['Prokka_annot'], hmm_db_path)
                if prokka_hmm_output_path is None:
                    print(f"Skipping phmmer due to missing HMM file for Prokka")
                    return None
                out_path = os.path.join(output_dir, f"{row['Prokka_annot']}_{row['BATH_annot']}_results.out")
                protein_sequence_path = os.path.join(output_dir, f"{row['BATH_annot']}_forphmmer.fa")
                with open(protein_sequence_path, 'w') as protein_file:
                    protein_file.write(f">{row['BATH_annot']}\n{row['BATH Uniprot Protein Sequence']}\n")
                cmd = ['phmmer', '--tblout', out_path, protein_sequence_path, prokka_hmm_output_path]
                result = subprocess.run(cmd, capture_output=True, text=True)

            elif row['BATH_annot'].startswith('MF_'):
                bath_hmm_output_path = fetch_onlyhmm_path(row['BATH_annot'], hmm_db_path)
                if bath_hmm_output_path is None:
                    print(f"Skipping phmmer due to missing HMM file for Bath")
                    return None
                out_path = os.path.join(output_dir, f"{row['Prokka_annot']}_{row['BATH_annot']}_results.out")
                protein_sequence_path = os.path.join(output_dir, f"{row['Prokka_annot']}_forphmmer.fa")
                with open(protein_sequence_path, 'w') as protein_file:
                    protein_file.write(f">{row['Prokka_annot']}\n{row.get('Prokka Uniprot Protein Sequence', '')}\n")
                cmd = ['phmmer', '--tblout', out_path, protein_sequence_path, bath_hmm_output_path]
                result = subprocess.run(cmd, capture_output=True, text=True)

            if result and result.returncode != 0:
                print(f"Error running phmmer: {result.stderr}")
                return None
            return result.stdout
    finally:
        # Clean up created files to prevent clutter
        if 'protein_sequence_path' in locals() and os.path.exists(protein_sequence_path):
            os.remove(protein_sequence_path)

output_dir = "/content/"

# Assuming df3 is a DataFrame that has been properly prepared with the necessary columns.
df3['phmmeroutput'] = df3.apply(lambda row: run_phmmer_HAMAP_Uniprot(row, output_dir, hmm_db_path), axis=1)
'''

## Plot E-values

In [ ]:
#BATH and Prokka E values
#Combine E values for Prokka in a single column
#Now plot a paired boxplot

In [23]:
df3.columns

Index(['BATH', 'BATH_annot', 'BATH_contig', 'BATH_end', 'BATH_start', 'Prokka',
       'Prokka_annot', 'Prokka_contig', 'Prokka_end', 'Prokka_start', 'label',
       'target name', 'query name', 'E-value', 'shifts', 'Prokka DNA Sequence',
       'Prokka Uniprot Protein Sequence', 'BATH Uniprot Protein Sequence',
       'Prokka_wbathsearch_HAMAP', 'Prokka_wbathsearch_HAMAP_E-value',
       'Prokka_wbathsearch_UniProt', 'Prokka_wbathsearch_UniProt_Evalue'],
      dtype='object')

In [24]:
df3[['Prokka_wbathsearch_HAMAP_E-value',
       'Prokka_wbathsearch_UniProt_Evalue']].to_excel("/content/check2.xlsx")

In [25]:
import pandas as pd

# Replace blank spaces with NaN
df3['Prokka_wbathsearch_HAMAP_E-value'] = df3['Prokka_wbathsearch_HAMAP_E-value'].replace('', pd.NA)
df3['Prokka_wbathsearch_UniProt_Evalue'] = df3['Prokka_wbathsearch_UniProt_Evalue'].replace('', pd.NA)

# Convert all entries to numeric, forcing non-convertible ones to NaN
df3['Prokka_wbathsearch_HAMAP_E-value'] = pd.to_numeric(df3['Prokka_wbathsearch_HAMAP_E-value'], errors='coerce')
df3['Prokka_wbathsearch_UniProt_Evalue'] = pd.to_numeric(df3['Prokka_wbathsearch_UniProt_Evalue'], errors='coerce')

# Use combine_first to combine the columns
df3['Combined_E-value'] = df3['Prokka_wbathsearch_HAMAP_E-value'].combine_first(df3['Prokka_wbathsearch_UniProt_Evalue'])

# Now df['Combined_E-value'] has the combined E-values
print(df3['Combined_E-value'])


0       1.300000e-91
1                NaN
2                NaN
3                NaN
4                NaN
           ...      
947              NaN
948    1.200000e-117
949     4.400000e-76
950              NaN
951     1.300000e-60
Name: Combined_E-value, Length: 952, dtype: float64


In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming df3 is your DataFrame
df3['Prokka_wbathsearch_Evalue'] = df3['Prokka_wbathsearch_Evalue'].astype(str).replace('None', '', regex=True)
df3['Prokka_wbathsearch_Evalue'] = pd.to_numeric(df3['Prokka_wbathsearch_Evalue'], errors='coerce')

# Convert E-values to numeric, coerce errors to NaN, and then drop rows with NaNs in these columns
df3[['Prokka_wbathsearch_Evalue','E-value','label']]



KeyError: 'Prokka_wbathsearch_Evalue'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Replace 0s with a small value (e.g., 1e-10) to avoid taking log(0)
df3['Prokka_wbathsearch_Evalue'] = df3['Prokka_wbathsearch_Evalue'].replace(0, 1e-10)
df3['E-value'] = df3['E-value'].replace(0, 1e-10)

# Log-normalize the data
df3['log_Prokka_wbathsearch_Evalue'] = np.log(df3['Prokka_wbathsearch_Evalue'])
df3['log_E-value'] = np.log(df3['E-value'])

# Create the scatter plot
sns.scatterplot(x='log_Prokka_wbathsearch_Evalue', y='log_E-value', hue='label', data=df3)
plt.xlabel('Log of Prokka_wbathsearch_Evalue')
plt.ylabel('Log of E-value')
plt.title('Scatter Plot of Log-Normalized E-values')
plt.show()


In [ ]:
df3.to_excel("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Organized/Evalues_BATH_ProkkawBATH_Fin.xlsx")

In [ ]:
df3[['Prokka_wbathsearch_Evalue','E-value','label']].to_excel("/content/1fjidjid.xlsx")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


df3 = pd.read_excel("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Organized/Evalues_BATH_ProkkawBATH_Fin.xlsx")
# Apply log normalization
df3['Prokka_log_E-value'] = np.log10(df3['Combined_E-value'])
df3['BATH_log_E-value'] = np.log10(df3['E-value'])

# Melt the DataFrame for plotting
melted_df = pd.melt(df3, id_vars='label', value_vars=['Prokka_log_E-value', 'BATH_log_E-value'],
                    var_name='Variable', value_name='Log_Evalue')

# Plot the grouped boxplots
plt.figure(figsize=(12, 10))
sns.violinplot(x='label', y='Log_Evalue', hue='Variable', split=True,  data=melted_df)
sns.stripplot(x='label', y='Log_Evalue', hue='Variable', dodge=True, color='grey', alpha=0.4, data=melted_df)
plt.xlabel('Label')
plt.ylabel('Log 10 Normalized E-value')
plt.title('Comparing Log Normalized E-values by Label after running BATH on Prokka Subsets')
plt.legend(title='E-value Type')
plt.xticks(rotation=45)  # Rotate x-axis labels if they overlap
ylim = plt.gca().get_ylim()
plt.yticks(np.arange(start=ylim[0], stop=ylim[1]+15, step=15))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Filter the DataFrame for rows where label is 'Prokka Annotated'
prokka_annotated = df3[df3['label'] == 'Prokka Annotated']


plt.figure(figsize=(10, 6))
plt.hist(prokka_annotated['Prokka_log_E-value'], bins=100, alpha=0.5, label='Log Combined E-value')

plt.xlabel('Log 10 Normalized E-value')
plt.ylabel('Frequency')

# Calculate x-ticks
data_min = prokka_annotated['Prokka_log_E-value'].min()
data_max = prokka_annotated['Prokka_log_E-value'].max()
# Add a vertical line for the mean
# Calculate the mean and median of the log values
mean_value = prokka_annotated['Prokka_log_E-value'].mean()
median_value = prokka_annotated['Prokka_log_E-value'].median()
plt.axvline(mean_value, color='r', linestyle='dashed', linewidth=1, label=f'Mean: {mean_value:.2f}')

# Add a vertical line for the median
plt.axvline(median_value, color='g', linestyle='dotted', linewidth=1, label=f'Median: {median_value:.2f}')

plt.xticks(np.arange(start=np.floor(data_min)-4, stop=np.ceil(data_max) + 12, step=4))

plt.title('Log Normalized E-values for Prokka Annotated')
plt.legend()
plt.show()


In [ ]:
#Plotting E-values for the COG proteins

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


import pandas as pd

# Read the first Excel file
df1 = pd.read_excel("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Organized/Evalues_BATH_ProkkawBATH_Fin.xlsx")

# Read the second Excel file
df2 = pd.read_excel("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Organized/phmmer_Evalues_COG_Matches.xlsx")

# Merge the DataFrames on 'Prokka' and 'BATH' columns
merged_df = pd.merge(df1, df2, on=['Prokka', 'BATH'], how='inner')

# Save the merged DataFrame to a new Excel file
merged_df.to_excel("/content/drive/MyDrive/Lab Work/Parkinsons_Data/Organized/merged_phmmer_Evalues.xlsx", index=False)


In [ ]:
merged_df['Prokka_log_E-value'] = np.log10(merged_df['Combined_E-value'])
merged_df['BATH_log_E-value'] = np.log10(merged_df['E-value_x'])

In [ ]:
merged_df.columns

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Melt the DataFrame for plotting
melted_df = pd.melt(merged_df, id_vars='Name_or_COG_Match', value_vars=['Prokka_log_E-value', 'BATH_log_E-value'],
                    var_name='Variable', value_name='Log_Evalue')

plt.figure(figsize=(12, 10))
sns.violinplot(x='Name_or_COG_Match', y='Log_Evalue', hue='Variable', data=melted_df, split=True, inner='quartile')
sns.stripplot(x='Name_or_COG_Match', y='Log_Evalue', hue='Variable', data=melted_df, dodge=True, color='k', alpha=0.5, jitter=True)
plt.title('Comparison of Log E-values')
plt.xlabel('Category')
plt.ylabel('Log E-value')
ylim = plt.gca().get_ylim()
plt.yticks(np.arange(start=ylim[0], stop=ylim[1]+25, step=25))
plt.xticks(['Yes', 'No'], ['Matching COG/Protein Annotation','Different COG/Protein Annotation'])
plt.legend(title='Variable')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()
